# Inverse Kinematics and PID control
In this homework assignemnt, you will be tasked with creating an IK solver and controlling a robot arm using PID control (specifically the Franka Panda 7DoF robot). We will be creating two versions of the controller, one that uses pure velocity control and another that using torque control with feedforward dynamics. 

### Import mujoco and numpy

In [1]:
import mujoco
import mujoco_viewer
import numpy as np
from numpy.linalg import inv, pinv
import scipy as sp

from utils import rot_err, getSitePose, getSiteJacobian, getMassMatrix, getBiasTerms, getOSMassMatrix
from autograder import autograder
from sim import Sim

sim = Sim(fname='./a1/xml/a1.xml')

In [2]:

class OperationalSpaceControl(object):
    def __init__(self, model, data) -> None:
        self._model = model
        self._data  = data 
        self._foot_site_names = [
            "FR_foot", "FL_foot", "RR_foot", "RL_foot"
        ]
        self._kp = 300.
        self._kd = 20.

    def __call__(self, ref_foot_pos):
        mass_matrix = getMassMatrix(self._model, self._data)
        bias        = getBiasTerms(self._model, self._data)
        tau = bias 

        for _foot_name in self._foot_site_names:
            (R_f, p_f) = getSitePose(self._model, self._data, _foot_name)
            jac_f      = getSiteJacobian(self._model, self._data, _foot_name)
            v_f        = jac_f @ self._data.qvel
            (R_d, p_d) = ref_foot_pos[_foot_name]

            err_p   = p_d - p_f
            err_rot = rot_err(R_d, R_f)
            err     = np.concatenate([err_p, err_rot])

            ### calculate the xddot_des in operational space 
            xddot_des = self._kp * err + self._kd *(-v_f)

            ### calculate the operational space mass matrix 
            op_mass_matrix = getOSMassMatrix(mass_matrix, jac_f)

            tau += inv(mass_matrix) @ jac_f.T @ op_mass_matrix @ xddot_des

        return tau
        

In [3]:
ctrlr = OperationalSpaceControl(sim.model, sim.data)
ref_foot_pos = {}
for _ft_name in ctrlr._foot_site_names:
    ref_foot_pos[_ft_name] = (np.eye(3), np.array([0., 0., 0.]))

In [4]:
t_max = 10000
render = True
if len(sim.model.key_qpos) != 0:
    sim.data.qpos = sim.model.key_qpos
mujoco.mj_step(sim.model, sim.data)
log = []
if render:
    viewer = mujoco_viewer.MujocoViewer(sim.model, sim.data)
    # viewer.vopt.frame = 3
for t in range(t_max):
    for _ft_name in ctrlr._foot_site_names:
        ref_foot_pos[_ft_name] = (np.eye(3), np.array([np.sin(t*0.01), 0., 0.]))
    tau = ctrlr(ref_foot_pos)
    sim.data.ctrl = tau
    mujoco.mj_step(sim.model, sim.data)
    if render:
        if viewer.is_alive:
            viewer.render()
if render:
    viewer.close()

/home/anon/miniconda3/envs/meng443arm/lib/python3.10/site-packages/glfw/__init__.py:906: GLFWError: (65544) b'Wayland: Standard cursor shape unavailable'
  warnings.warn(message, GLFWError)


Pressed ESC
Quitting.


KeyboardInterrupt: 